In [ ]:
%run preprocecssing_helpers.ipynb
import os

import mne
from mne.preprocessing import ICA
import numpy
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = (50.0, 30.0)
matplotlib.rcParams.update({'font.size': 15})

In [ ]:
path = 'E:\StroopEEG'
eeglab_files = os.listdir(path)

In [ ]:
eeglab_dict = search_folders(path, eeglab_files)
eeglab_files = paths_of_eeglab_files(path, eeglab_dict)
eeglab_files

In [ ]:
eeglab_files[0]

In [ ]:
# read file
raw = mne.io.read_raw_eeglab(eeglab_files[1], preload=True)

In [ ]:
picks = mne.pick_types(raw.info, meg = False, eeg= True, eog= False, stim = False)


In [ ]:
raw.plot();



In [ ]:
raw.notch_filter(np.arange(60,241,60), filter_length='auto', phase = 'zero')
raw.filter(5,100, fir_design='firwin')

In [ ]:
ica = ICA(n_components=None, max_iter = 1000, random_state = 42)

In [ ]:
ica.fit(raw,picks = None, decim=3, reject=dict(mag=4e-12, grad=4000e-13))

In [ ]:
ica.detect_artifacts(raw,start_find=None, stop_find=None, ecg_ch=None, 
                     ecg_score_func='pearsonr', skew_criterion=None, 
                     kurt_criterion=None, var_criterion=None,add_nodes=None)

In [ ]:
ica.plot_sources(raw);

In [ ]:
ica.plot_overlay(raw, start = 0, stop = 100);

In [ ]:
raw_ica = ica.apply(raw)

In [ ]:
raw_ica.plot(duration=20,  n_channels=128);

In [ ]:
# clean data
raw.filter(1, None, n_jobs= 10, fir_design= 'firwin')
picks_eeg = mne.pick_types(raw.info, eeg = True, meg = False, eog = False, exclude= 'bads')
method = 'fastica'
n_components= 25
decim = 3
random_state = 23

In [ ]:
ica = ICA(n_components= n_components, method = method, random_state= random_state)
print(ica)

In [ ]:
reject = dict(mag = 5e-12, grad = 4000e-13)
ica.fit(raw, decim = decim, reject = reject)

In [ ]:
ica.detect_artifacts(raw) # can you spot some potential bad guys?

In [ ]:
try: 
    a,b = ica.find_bads_eog(raw)
    ica.exclude += a
except:
    pass

* Can't plot results due to type mismatching and missing electrode locations.

In [ ]:
ica.score_sources

In [ ]:
raw_avg = mne.set_eeg_reference(raw, 'average', projection= False)